<h1>Part B (Neural Network as Logic Gates)</h1>

In [1681]:
# Imports
import math
import torch
import numpy as np

<h3>Class Definition</h3>

In [1682]:
class NeuralNetwork():
    
    # When we initialize a neural network the only information that we give it 
    # will be a list of the size of each layer in the network.
    def __init__(self, layer_sizes):
        
        # Get the number of layers that the user wants the network to contain.
        self.layer_sizes = layer_sizes
        self.num_layers = len(self.layer_sizes)
        
        # Build the weights for our network.
        # The layout for the weights will be a list of matrices for now.
        # The shape of a given weight item in the list will be [from layer size + 1, to layer size]
        # We add the plus 1 in the from size to account for the bias neuron of that layer.
        self.thetas = [0 for x in range(0,len(layer_sizes)-1)]
            
    # For the NeuralNetwork class getLayer() just sends the caller the list of weights
    # for the entire network.
    def getLayer(self, layer):
        return self.thetas[layer]
        
    # Where the magic happens for this class. When the user calls the forward method 
    # for a NeuralNetwork object with an input list of integers, we feed those
    # integers into the network, multiplying by each weight matrix.
    def forward(self, input_list):

        # Initialize a list of all of the weighted sums that we will get from 
        # multiplying a previous layer by the connection weights.
        z = []
        
        # Initialize a list of all of the sigmoid nonlinearity results
        # This will end up being the same size as the weighted sums (z)
        a = []
        
        # This is just a PyTorch tensor created from the users' input and 
        # reshaped to be a horizontal row of inputs.
        first_layer_no_bias = torch.FloatTensor(input_list).view(1, len(input_list))
        
        # Add our first z values which will be the input layer. Since there is no nonlinearity
        # applied to the input before sending it into the network, we just set the first
        # a value to be the input array as well.
        z.append(first_layer_no_bias)
        a.append(z[0])
                    
        # Iterate through each layer of the matrix until 1 before the output layer.
        for i in range(0,self.num_layers-1):
            
            # Create a PyTorch tensor with a single value and then concatenate it 
            # onto the front of the input array. This is our bias value and when 
            # concatenated it creates our input along with the bias. 
            # We add a bias item to every layer within this loop
            bias = torch.ones(1, 1)
            layer_with_bias = torch.cat([bias, a[i]], 1)
            
            # Create the input to our sigmoid nonlinearity by matrix multiplying 
            # the current layer that we're on with the appropriate weight matrix.
            z.append(torch.mm(layer_with_bias, self.thetas[i]))
            a.append(1/(1+np.exp(-z[-1])))
            

        # This is a bit of a bug, I'm getting outputs that are not in fact shaped according 
        # to any laws of linear algebra. For now just taking the first element of the very last
        # a value has been doing just fine. 
        result = a[-1][0][0]
        
        # If binary output desired:
        if result >= 0.5:
            result = 1.0
        elif result < 0.5:
            result = 0.0
        
        return result

<h2>Logic Gate Class Definition</h2>

In [1683]:
class AND():
    def __init__(self):
        layer_sizes = [2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x, y):
        self.x = int(x)
        self.y = int(y)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x, self.y]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1684]:
class OR():
    def __init__(self):       
        layer_sizes = [2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x, y):
        self.x = int(x)
        self.y = int(y)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x, self.y]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1685]:
class NOT():
    def __init__(self):
        layer_sizes = [2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x):
        self.x = int(x)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x]
        result = self.network.forward(input_array)
        
        return bool(result)

In [1686]:
class XOR():
    def __init__(self):
        layer_sizes = [2, 2, 1]
        self.network = NeuralNetwork(layer_sizes)

    def __call__(self, x, y):
        self.x = int(x)
        self.y = int(y)
        return self.forward()
    
    def getLayer(self, theta, layer):
        self.network.thetas[layer] = theta
        
    def forward(self):
        input_array = [self.x, self.y]
        result = self.network.forward(input_array)
        
        return bool(result)

<h3>Class Instantiation</h3>

In [1687]:
And = AND()
Or = OR()
Not = NOT()
Xor = XOR()

AND result:  True
OR result:  True
NOT result:  True
XOR result:  False


<h2>Weight Definitions</h2>

<h3>AND Weights</h3>

In [ ]:
and_thetas = [torch.tensor([
    [-2.0], 
    [ 1.5], 
    [ 1.5]
]).float()]

<h3>OR Weights</h3>

In [ ]:
or_thetas = [torch.tensor([
    [-0.25], 
    [ 1.5], 
    [ 1.5]
]).float()]

<h3>AND Weights</h3>

In [ ]:
not_thetas = [torch.tensor([
    [ 0.0],
    [-1.0]
]).float()]

<h3>XOR Weights</h3>

In [ ]:
xor_thetas = [torch.tensor([
    [-0.25, 2.0],
    [ 1.5, -1.5],
    [ 1.5, -1.5]
]).float(), torch.tensor([
    [-2.0],
    [ 1.5],
    [ 1.5]
]).float()]

<h3>getLayer() Method Testing/Set Weights<h3>

In [ ]:
And.getLayer(and_thetas[0], 0)
Or.getLayer(or_thetas[0], 0)
Not.getLayer(not_thetas[0], 0)
Xor.getLayer(xor_thetas[0], 0)
Xor.getLayer(xor_thetas[1], 1)

<h3>Testing The Logic Gates</h3>

In [ ]:
print("AND result: ", And(True, True))
print("OR result: ", Or(False, True))
print("NOT result: ", Not(False)) 
print("XOR result: ", Xor(False, False))